In [27]:
# Load env variables and create client
import os
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()


client = Anthropic(
  api_key=os.getenv("ZHIPU_API_KEY"),
  base_url="https://open.bigmodel.cn/api/anthropic"
)

model = "glm-4.7"

In [28]:
# Helper functions
from anthropic.types import Message

def add_user_message(messages, message):
    user_message = {
        "role": "user", 
        "content": message.content if isinstance(message, Message) else message
    }
    messages.append(user_message)

def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant", 
        "content": message.content if isinstance(message, Message) else message
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message

def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == 'text']
    )

In [29]:
from datetime import datetime
from anthropic.types import ToolParam
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot empty")
    return datetime.now().strftime(date_format)



get_current_datetime_schema = ToolParam({
    "name": "get_current_datetime",
    "description": "Returns the current date and time formatted according to the specified format",
    "input_schema": {
        "type": "object",
        "properties": {
            "date_format": {
                "type": "string",
                "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes.",
                "default": "%Y-%m-%d %H:%M:%S"
            }
        },
        "required": []
    }
})



In [33]:
import json


def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)


def run_tools(message):
    tool_requests = [block for block in message.content if block.type == "tool_use"]
    tool_result_blocks = []

    for tool_request in tool_requests:
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks

In [36]:
def run_conversation(messages):
    while True:
        response = chat(
            messages,
            tools=[
                get_current_datetime_schema,
            ],
        )

        add_assistant_message(messages, response)
        print(text_from_message(response))

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

    return messages

In [37]:
messages = []
add_user_message(messages, "what is the exact time, formatted as HH:MM:SS? also what is current date formatted as YYYY-MM-DD?")
run_conversation(messages)


Here's the current time and date in your requested formats:

- **Current time (HH:MM:SS):** 22:48:51
- **Current date (YYYY-MM-DD):** 2026-01-26


[{'role': 'user',
  'content': 'what is the exact time, formatted as HH:MM:SS? also what is current date formatted as YYYY-MM-DD?'},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='call_88333da3bcce48baa34a366c', input={'date_format': '%H:%M:%S'}, name='get_current_datetime', type='tool_use'),
   ToolUseBlock(id='call_a38e6ae3df4a4e6e8eda265a', input={'date_format': '%Y-%m-%d'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'call_88333da3bcce48baa34a366c',
    'content': '"22:48:51"',
    'is_error': False},
   {'type': 'tool_result',
    'tool_use_id': 'call_a38e6ae3df4a4e6e8eda265a',
    'content': '"2026-01-26"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text="Here's the current time and date in your requested formats:\n\n- **Current time (HH:MM:SS):** 22:48:51\n- **Current date (YYYY-MM-DD):** 2026-01-26", type='text')]}]